In [ ]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# Libraries and functions

In [1]:
import numpy as np
import pandas as pd
import pandasql as ps
from pandas import Series, DataFrame
import datetime as dt
import matplotlib.pyplot as plt
# import matplotlib as mpl
import sys
import time
import math
from tqdm import tqdm
from importlib import reload
from os.path import join, normpath
from bs4 import BeautifulSoup
import os
from os import path
import roamability as rb
import re

downloads = 'C:/Users/balob/Downloads'

In [99]:
%reset -f

# Examples

### MS SQL request

In [208]:
sql_srt='SELECT COUNT(*) FROM ACCOUNTS;'
with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
    df = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)

### DMI Oracle request

In [211]:
sql_srt = 'SELECT * FROM SPONSOR'
# with rb.OracleConnect('DMI', 'dd607605ce341', 'DMI_TEST') as cnxn:
with rb.OracleConnect('DMI', 'dd607605ce341', 'DMI') as cnxn:
    df = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)

### BSS MySql request

In [1096]:
sql_srt='SELECT MSISDN, VisitedNetworkTadig FROM TAP.GPRS_CALL LIMIT 5'
with rb.MySqlConnect('172.18.11.40', 'BSS', 'noc', 'WcQUzkXiXwoxnFfGnRxb') as cnxn:
    df = pd.read_sql_query(sql_srt, cnxn)
df

,MSISDN,VisitedNetworkTadig
0,447700095423,BRAV1
1,972541033641,BRAV1
2,972541039874,BRAV1
3,972541033641,BRAV1
4,447700095423,BRAV1


### Soap API OCS Portal

In [163]:
# %reset -f
from zeep import Client
from zeep.wsse.username import UsernameToken

connect = input("Choose connection (prod or lab):")

if connect == 'lab':
    user = 'lab_api@roamability.com'
    user = 'api_lab@dialoq.com'
    password = '25D55AD283AA400AF464C76D713C07AD' #Lab
    password = 'FAC4EC08B5FC0CCF8145BB79E8F24F47' #Lab
    api_link = 'https://172.20.39.7:8585/ocsapi/roamability/api/roamability.wsdl' #Lab
    api_link = 'https://labocsapi.roamability.com:6443/ocsapi/roamability/api/roamability.wsdl'
    api_link = 'http://172.20.39.7:8585/ocsapi/roamability/api/roamability.wsdl' #Lab
elif connect == 'prod':
    user = 'prod_api@roamability.com'
    password = '25D55AD283AA400AF464C76D713C07AD'
    api_link = 'https://ocsapi.roamability.com:7443/ocsapi/roamability/api/roamability.wsdl'
#     api_link = 'https://ocsapi.roamability.com/ocsapi/roamability/api/roamability.wsdl'
#     api_link = 'https://ocsapi.roamability.com:7443/ocsapi/roamability/api/'
#     api_link = 'https://ocsapi2.roamability.com:7443/ocsapi/roamability/api/roamability.wsdl'
#     api_link = 'http://172.20.35.10:8585/ocsapi/roamability/api/roamability.wsdl'
#     api_link = 'http://172.20.35.12:8585/ocsapi/roamability/api/roamability.wsdl'

user_name_token = UsernameToken(user, password)
user_name_token.use_digest = True
client = Client(api_link, wsse=user_name_token)

reseller = client.service.getResellerInfo()
accounts = client.service.getAccounts()

if reseller.result.code == '1':
    print('You are connected to Reseller: {}.\
    \nAvailable credit for the Reseller: {:.0f} USD.\n'.\
    format(reseller.reseller.resellerName,reseller.reseller.availableCredit))
else:
    print('Error in getting Reseller info: {}'.format(reseller.result.description))

if accounts.result.code == '1':
    accounts_dict = {account.accountId:account.accountName for account in accounts.accounts.accounts}
    print('The Reseller contains the following accounts:')
    for i,[account_id,account_name] in enumerate(accounts_dict.items()):
        print(' {:<5}{} (accountID: {})'.format(i,account_name,account_id))
else:
    print('Error in getting Account info: {}'.format(accounts.result.description))

# Subscriber in lab in STI account
#subscriber_id = client.service.getSubscriberById('934420')
#subscriber_iccid = client.service.getSubscriberByICCID('8997219121000031446')
# client.service.getSubscriberPrePaidPackages('831882', 1)
#client.service.getSubscriberByIMSI('425019613023160')
#client.service.getSubscriberById('209287')
#client.service.getSubscriberByICCID('8997219121000022660')
# client.service.assignSubscriberPrePaidPackage('831882', 1, dt.datetime.strptime('2019-07-07','%Y-%m-%d'))
# client.service.getSubscriberPrePaidPackages('937939', 1)
# client.service.expireSubscriberPrePaidPackage(1848972,60311)
# client.service.getSubscriberPrePaidPackages(1848972,1)
# prices = client.service.getMobilePrices() # prices.destinations[0]
# client.service.getPricingPlans()
# client.service.modifySubscriberSpeedLimit(1449606, '256kbps')
# client.service.modifySubscriberSpeedLimit(210000, '1024kbps') # lab
#subscriber_id

Choose connection (prod or lab):prod
You are connected to Reseller: Dialoq - Genesisclub RT.    
Available credit for the Reseller: 8085 USD.

The Reseller contains the following accounts:
 0    Genesisclub Test sim cards (accountID: 352475)
 1    Genesisclub Partner Soft test (accountID: 352642)
 2    Genesisclub P4 Soft test (accountID: 352643)
 3    Truphone S1 Soft (accountID: 352770)
 4    Truphone S2 Soft (accountID: 352771)
 5    Truphone QR DATA S2+S1+S4 (accountID: 352820)
 6    Dailoq S2+S1+S4 (accountID: 352827)
 7    New Profiles (accountID: 352857)
 8    new applet profile test (accountID: 352858)
 9    Truphone QR DATA_CALLS S2+S1+S4 (accountID: 352859)
 10   dialoq_10K_191119 (accountID: 352891)
 11   Truphone S5 Soft (accountID: 352954)
 12   Truphone S8 Soft (accountID: 352955)
 13   Dailoq_200522_5K_S2_S1 (accountID: 352979)


### Send MSU

 ### Format output

In [ ]:
import datetime as dt
print( 'start: {} {} :end'.format('one', 'two') ) #start: one two :end
print( 'start: {1} {0} :end'.format('one', 'two') ) #start: two one :end
print( 'start: test {:10} :end'.format('test') ) #start: test test       :end
print( 'start: test {:10} :end'.format(10) ) #start: test         10 :end
print( 'start: test {:>10} :end'.format('test') ) #start: test       test :end
print( 'start: test {:<10} :end'.format('test') ) #start: test test       :end
print( 'start: test {:_<10} :end'.format('test') ) #start: test test______ :end
print( 'start: test {:^10} :end'.format('test') ) #start: test    test    :end
print( 'start: test {:.10} :end'.format('test1test2test3') ) #start: test test1test2 :end
print( 'start: test {:_^10.5} :end'.format('test1test2test3') ) #start: test __test1___ :end
print( 'start: test {:d} :end'.format(42) ) #start: test 42 :end
print( 'start: test {:f} :end'.format(42) ) #start: test 42.000000 :end
print( 'start: test {:06.2f} :end'.format(3.141592) ) #start: test 003.14 :end
print( 'start: test {p[first]} {p[last]} :end'.format(p={'first': 'Jean-Luc', 'last': 'Picard'}) ) #start: test Jean-Luc Picard :end
print( 'start: test {p[4]} {p[5]} :end'.format(p=[4, 8, 15, 16, 23, 42]) ) #start: test 23 42 :end
print( 'start: test {:%Y-%m-%d %H:%M} :end'.format(dt.datetime(2001, 2, 3, 4, 5)) ) #start: test 2001-02-03 04:05 :end

### Search in files by Tags

In [233]:
import roamability as rb
reload(rb)

# Где искать файлы .txt с тегами
paths = ['c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/NOC_HOW_IT_WORKS/',
        'c:/W_DATA_ROAM/МЕТОДИКИ/',
        'c:/W_DATA_ROAM/ПРОЕКТЫ/',
        'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/NOC_KNOWN_ISSUES/',
        'c:/W_DATA_ROAM/МЕТОДИКИ/_NOC_M_NEW_CONNECTION/',
        'c:/W_DATA_ROAM/ALARMS/',
        'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/PRICES',
        ]

# Если хотя бы один из тэгов присутствует. Нужно указать минимум один.
# any_tags  = ['#reports_bss_usage_mop']
# any_tags  = ['#gy_principle_of_operation']
# any_tags  = ['#ocs_principle_of_operation']
# any_tags  = ['#reports_ocs_usage_mop']
# any_tags  = ['#reports_ocs_subscribers']
# any_tags = ['#epg_principle_of_operation']
any_tags  = ['#reports_ocs_subscribers']
# any_tags  = ['#pbx_principle_of_operation']

# Если все теги присутствуют. Нужно оставить лист пустым [] если нет обязательных тэгов.
all_tags = []

rb.find_files_by_tags(paths, all_tags, any_tags)

****************************************************************************************************
c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\NOC_HOW_IT_WORKS\REPORT_OCS_Subscribers.txt
----------------------------------------------------------------------------------------------------
#reports_ocs_subscribers
------------------------------
Отчет выводит абонентов с привязкой к аккаунту и реселлеру

****************************************************************************************************
c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\NOC_HOW_IT_WORKS\REPORT_Resellers_Accounts_Subscribers_A.txt
----------------------------------------------------------------------------------------------------
#reports_ocs_subscribers
------------------------------
Отчет Реселлер\Аккаунт\PP\Кол-во абонентов
Отчет Реселлер\Аккаунт\Абонент (IMSI, MSUSDN)\Prepaid package



### Search in ALARMS by folder names

In [592]:
import os

paths = [
         r'c:/W_DATA_ROAM/ALARMS/',
         r'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/PRICES/',
         r'c:/W_DATA_ROAM/ПРОЕКТЫ/',
         r'c:/W_DATA_ROAM/ПРОЕКТЫ_NEW_CONNECTION/',
    r'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ',
    r'c:\W_DATA_ROAM_DOC\ПРОЕКТЫ',
        ]

for path in paths:
    names = os.listdir(path)
    [print(normpath(path + name)) for name in names if name.upper().find('S6_')!=-1]

c:\W_DATA_ROAM\ALARMS\MOC_AREDU_S6_no_CAMEL_200727_#4900471
c:\W_DATA_ROAM\ALARMS\MOC_S6_GBRVF_200716
c:\W_DATA_ROAM\ALARMS\MOC_S6_PSEJE_200628
c:\W_DATA_ROAM\ALARMS\PS_S6_AREDU_200702
c:\W_DATA_ROAM\ALARMS\UL_S6_ISR01_SS7_200625
c:\W_DATA_ROAM\ПРОЕКТЫ\200215_GY_RELAY_S5_S6_S8
c:\W_DATA_ROAM\ПРОЕКТЫ\200422_PP_S6_OCS
c:\W_DATA_ROAM\ПРОЕКТЫ\200425_AL_S6_ALL
c:\W_DATA_ROAM\ПРОЕКТЫ\200701_PP_S6_OCS
c:\W_DATA_ROAM_DOC\ПРОЕКТЫ191109_SOFT_S6_S8
c:\W_DATA_ROAM_DOC\ПРОЕКТЫ191128_10K_S1_S2_S4_S5_S6_S8_DrimSim


### Search in Allowed Lists and others

In [700]:
downloads = 'C:/Users/balob/Downloads'
dmi_allowed_list = 'DMI.dmi_allowed_list_export_Sun_Aug_02_2020.csv'
dmi_netpfx = 'DMI.dmi_netpfx_export_Wed_Jun_03_2020.csv'
bss_usage = 'bss_usage_2019_08.csv' # c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\NOC_HOW_IT_WORKS\BSS_SQL\Report_BSS_Usage_per_PLMN.txt

#Like expression for Allowed List
df_al=pd.read_csv(join(downloads, dmi_allowed_list))
df_al['SPONSOR'] = df_al['Sponsor'].replace(to_replace={r'[Pp][Aa][rtner]?[\s\w-]*|^S1_[\s\w-]*':'S1',
                                                        r'[Pp]4[\s\w-]*|^S2_[\s\w-]*':'S2',
                                                        r'[Mm][Bb][\s\w-]*|^S4_[\s\w-]*':'S4',
                                                        r'SMART[\s\w-]*|^S5_[\s\w-]*':'S5',
                                                        r'^S8_[\s\w-]*':'S8'}, regex=True)
df_al.rename({'PLMN code':'TADIG'}, axis=1, inplace=True)

df_usage = pd.read_csv(join(downloads, bss_usage), sep=';')
cols = ['REP_DATE','VPLMN','SPONSOR','MOC','MOC_SMS','DATA']
df_usage = df_usage.pivot_table(values='COST', index=['REP_DATE', 'VPLMN', 'SPONSOR'], columns='USAGE_TYPE').reset_index()[cols]
df_usage.rename({'VPLMN':'TADIG'}, axis=1, inplace=True)
df_usage.columns.name=''

df_al = pd.merge(df_al, df_usage, how='left', on=['SPONSOR','TADIG'])

df_np=pd.read_csv(join(downloads, dmi_netpfx))

In [17]:
df_al.loc[(df_al['TADIG'].str.startswith(tadig, na=False)) &
          (df_al['Sponsor'].str.contains(sponsor, na=False, regex=True, flags=re.I)),
          ['Sponsor','TADIG','REP_DATE','MOC','MOC_SMS','DATA','SPONSOR']].sort_values(by=['Sponsor','TADIG']).SPONSOR.unique()

array(['S2', 'S1'], dtype=object)

In [701]:
sponsor = '' # Use regular expressions if it's needed
tadig = 'ARETC'

df_al.loc[(df_al['TADIG'].str.startswith(tadig, na=False)) &
          (df_al['Sponsor'].str.contains(sponsor, na=False, regex=True, flags=re.I)),
          ['Sponsor','TADIG','REP_DATE','MOC','MOC_SMS','DATA','SPONSOR']].sort_values(by=['Sponsor','TADIG'])

,Sponsor,TADIG,REP_DATE,MOC,MOC_SMS,DATA,SPONSOR
4179,MB_ALL,ARETC,NaN,NaN,NaN,NaN,S4
3377,MB_Combined,ARETC,NaN,NaN,NaN,NaN,S4
1508,MB_DIAL01_COMB,ARETC,NaN,NaN,NaN,NaN,S4
1479,MB_DIAL_COMB,ARETC,NaN,NaN,NaN,NaN,S4
1914,MB_TottComb,ARETC,NaN,NaN,NaN,NaN,S4
6033,SMART,ARETC,NaN,NaN,NaN,NaN,S5
5537,SMART_Tottol,ARETC,NaN,NaN,NaN,NaN,S5


In [147]:
df_al.loc[(df_al['TADIG'].str.startswith(tadig, na=False)) &
          (df_al['Sponsor'].str.contains(sponsor, na=False, regex=True, flags=re.I)),
          ['Sponsor','TADIG','SPONSOR']].sort_values(by=['Sponsor','TADIG']).Sponsor.unique()

array(['MB_DRIM01_ALL', 'P4_DRIM01_ALL', 'Partner_DRIM01_A',
       'S5_DRIM01_ALL'], dtype=object)

In [680]:
plmn = 'MDGCO'
df1 = df_al.loc[(df_al['TADIG'].str.startswith(plmn, na=False)) &
          (df_al['Sponsor'].str.contains('', na=False)), ['Sponsor','TADIG']].sort_values(by=['Sponsor','TADIG'])
df2 = df_np[(df_np.Operator.str.startswith(plmn,na=False))] #.to_csv(join(downloads,'Prefix.csv')) # & (df_np.SSN==7)
display(df1)
display(df2)

,Sponsor,TADIG
3210,PA_Telzar_Soft,MDGCO
201,Partner,MDGCO
1206,Partner_2018,MDGCO
2726,Partner_All,MDGCO
3262,Partner_Clay_Com,MDGCO
4111,Partner_Com_JWM,MDGCO
2384,Partner_Combined,MDGCO
4901,Partner_DIAL01_C,MDGCO
3858,Partner_DIAL_COM,MDGCO
5448,Partner_DRIM01_A,MDGCO


,Operator,Prefix,Number/Range,SSN,MCC,MNC
3034,MDGCO,26133,1,7,646,1
3035,MDGCO,26133,1,149,646,1
4460,MDGCO,2613311100,0,7,646,1
4461,MDGCO,2613311101,0,7,646,1
8038,MDGCO,2613311101,0,149,646,1
8963,MDGCO,26133111010,0,1,646,1


In [816]:
str(df_np[(df_np.Prefix.str.startswith('393519999',na=False))].Prefix.unique().tolist())
df_np[(df_np.Prefix.str.startswith('3774',na=False))]

,Operator,Prefix,Number/Range,SSN,MCC,MNC
81,MCOM1,37744,1,7,212,1
82,MCOM1,37744,1,149,212,1
83,MCOM1,37745,1,149,212,1
1164,MCOM1,37745,1,7,212,1
1165,MCOM1,37744010405,0,1,212,1
5382,MCOM2,3774000,1,7,212,10
5383,MCOM2,3774000,1,149,212,10
5384,MCOM2,3774098,1,149,212,10
5385,MCOM2,3774098,1,7,212,10
5386,MCOM2,3774000400,1,1,212,10


### Hex Bin Dec conversion

In [593]:
len('0114010301001E01140000000300030501020000000000000000000000000000000000000000000000000000000000'.replace(' ', '')) / 2

47.0

In [689]:
hex_int = 0x73
dec_int = 47
bin_int = 0b100
# Hex to dec
print('Hex to dec')
print(f'{hex_int:0>16d}')
# Hex to bin
print('Hex to bin')
print(f'{hex_int:0>16b}')
# Dec to hex
print('Dec to hex')
print(f'{dec_int:F>16X}')
# Dec to bin
print('Dec to bin')
print(f'{dec_int:0>16b}')
# Bin to hex
print('Bin to hex')
print(f'{bin_int:F>16X}')
# Bin to dec
print('Bin to dec')
print(f'{bin_int:0>16d}')

Hex to dec
0000000000000115
Hex to bin
0000000001110011
Dec to hex
FFFFFFFFFFFFFF2F
Dec to bin
0000000000101111
Bin to hex
FFFFFFFFFFFFFFF4
Bin to dec
0000000000000004


In [222]:
7-243-1
4-4-2

f'{bin(7)} - {bin(243)} - {bin(1)}'
#2.61.6

'0b111 - 0b11110011 - 0b1'

In [223]:
print(bin(10915))
a = '11111110011001'
# = '00000011001101'
print(int(a[-14:-11],2),'-',int(a[-11:-3],2),'-',int(a[-3:],2) )

0b10101010100011
7 - 243 - 1


### Send SMS via SMPP

In [593]:
import smpplib
import smpplib.gsm
client = None
#parts, encoding_flag, msg_type_flag = smpplib.gsm.make_parts(u'Hello world!')
parts, encoding_flag, msg_type_flag = smpplib.gsm.make_parts("""
Test message via STI SMPP supplier.
""")

# client = smpplib.client.Client('172.18.11.20', 2770) # Prod
client = smpplib.client.Client('172.18.11.90', 2770) # Lab

client.connect()
# client.bind_transceiver(system_id='test', password='test') # Prod
client.bind_transceiver(system_id='test', password='ChangeMe') # Lab

for part in parts:
    pdu = client.send_message(
        source_addr_ton=smpplib.consts.SMPP_TON_INTL,
#         source_addr_npi=smpplib.consts.SMPP_NPI_ISDN,
        # Make sure it is a byte string, not unicode:
        source_addr='17783003587',
#         source_addr='IsraelPostIsraelPost',
        dest_addr_ton=smpplib.consts.SMPP_TON_INTL,
        #dest_addr_npi=smpplib.consts.SMPP_NPI_ISDN,
        # Make sure thease two params are byte strings, not unicode:
#         destination_addr='972559300036',
        destination_addr='972544850997',
        short_message=part,
        data_coding=encoding_flag,
        esm_class=msg_type_flag,
        registered_delivery=True,)
    print(f'PDU SQN: {pdu.sequence}')
client.unbind()
client.unbind()
client.disconnect()

PDU SQN: 2


## Testing

In [747]:
import requests
ip_addr = '192.168.1.44'
port = 80
url_01 = f'http://{ip_addr}:{port}/climate;sensor=1&readattempt=10&temperature=25.0&humidity=35.0'
url_02 = f'http://{ip_addr}:{port}/climate;sensor=4&readattempt=10&temperature=25.0&humidity=35.0'
url_03 = f'http://{ip_addr}:{port}/gas;sensor=2&sensorValue=666'
url_04 = f'http://{ip_addr}:{port}/motion;sensor=3'
for url in [url_01, url_02, url_03, url_04]:
    r = requests.get(url)
    print(r.status_code)
    print(r.text.strip())

200
"OK"
200
"OK"
200
"OK"
200
"OK"


In [702]:
# Data
sdrRate = 1.405533
charge = 1199.0644
chargedUnits = 21944320
unit = 1024 * 1014
(charge * sdrRate / chargedUnits) * unit

79.74424300008178

In [458]:
# Sms
sdrRate = 1.37522
charge = 0.16527
chargedUnits = 1
unit = 1
(charge * sdrRate / chargedUnits) * unit

0.23303069999999998

In [460]:
# Voice
sdrRate = 1.405533
charge = 0.0282
chargedUnits = 106
unit = 60
(charge * sdrRate / chargedUnits) * unit

0.02243548901886792

### Developments

In [392]:
data_size = 8389594
time = 1
(data_size / time) / (1024 * 1024)

8.000940322875977

In [ ]:
# Отчет для Yehi:
# "I need the number of Active SIM cards with low ARPU."

In [757]:
start_date = dt.date(2020,7,1) # Including
end_date = dt.date(2020,7,31) # Including

sql_srt = """
SELECT
Reseller
,SubscriberId
,SUM(Revenue) AS Charge
FROM (SELECT
s.Customer AS Reseller
,ev.Subscriber AS SubscriberId
,ev.Revenue
FROM TAP.SUBSCRIBER s, TAP.MOC ev FORCE INDEX (ind_treated_moc)
WHERE ev.Subscriber = s.Id
AND ev.TreatedTimestampUtc BETWEEN {start_date} AND {end_date}
UNION ALL
SELECT
s.Customer
,ev.Subscriber AS SubscriberId
,ev.Revenue
FROM TAP.SUBSCRIBER s, TAP.GPRS_CALL ev FORCE INDEX (ind_treated_gprs)
WHERE ev.Subscriber = s.Id
AND ev.TreatedTimestampUtc BETWEEN {start_date} AND {end_date}) t
GROUP BY t.Reseller, t.SubscriberId
""".format(start_date = start_date.strftime("%Y%m%d") + '000000', end_date = end_date.strftime("%Y%m%d") + '235959')
with rb.MySqlConnect('172.18.11.40', 'BSS', 'noc', 'WcQUzkXiXwoxnFfGnRxb') as cnxn:
    df_bss_subs_charge = pd.read_sql_query(sql_srt, cnxn)

def cut_charge(x):
    if x <= 1:
        return 'ARPU <= 1 USD'
    elif x > 1 and x <= 5:
        return 'ARPU > 1 and <= 5 USD'
    elif x > 5 and x <= 10:
        return 'ARPU > 5 and <= 10 USD'
    elif x > 10:
        return 'ARPU > 10 USD'

df_bss_subs_charge['ARPU'] = df_bss_subs_charge.Charge.map(cut_charge)

df_bss_subs_charge = df_bss_subs_charge.pivot_table(
    values='SubscriberId',index='Reseller',columns='ARPU',aggfunc='count').reset_index()

df_bss_subs_charge

,Reseller,SubscriberId,Charge
0,Cloud 9 TAP,8,0.008511
1,Cloud 9 TAP,267,0.125750
2,Cloud 9 TAP,543,0.000318
3,Cloud 9 TAP,577,0.000363
4,Cloud 9 TAP,581,0.000416


ARPU,Reseller,ARPU <= 1 USD,ARPU > 1 and <= 5 USD,ARPU > 10 USD,ARPU > 5 and <= 10 USD
0,Cloud 9 TAP,6297.0,2396.0,591.0,686.0
1,DENT,372.0,67.0,63.0,32.0
2,Dialoq - Genesisclub RT,171.0,114.0,60.0,36.0
3,Drimsim,628.0,133.0,35.0,41.0
4,FirstPoint,1.0,NaN,NaN,NaN
5,Freecell,35.0,20.0,19.0,9.0
6,KnowRoaming IMSI-RANGE TAP,26639.0,546.0,118.0,88.0
7,MCN Telecom,5.0,NaN,1.0,NaN
8,Mobilise,3.0,NaN,NaN,NaN
9,Mondicon Pepperminds,456.0,1.0,NaN,NaN


In [737]:
# Step 1. Report from Splunk for July
# index=ocs Application="OCS_RRE" UsageType ResellerCharge | stats sum(ResellerCharge) AS Charge by Reseller,SubscriberId

df_splunk_subs_charge = pd.read_csv(join(downloads, 'ResellerSubscriberChargeJuly2020.csv'))

def cut_charge(x):
    if x <= 1:
        return 'ARPU <= 1 USD'
    elif x > 1 and x <= 5:
        return 'ARPU > 1 and <= 5 USD'
    elif x > 5 and x <= 10:
        return 'ARPU > 5 and <= 10 USD'
    elif x > 10:
        return 'ARPU > 10 USD'

df_splunk_subs_charge['ARPU'] = df_splunk_subs_charge.Charge.map(cut_charge)

df_splunk_subs_charge = df_splunk_subs_charge.pivot_table(
    values='SubscriberId',index='Reseller',columns='ARPU',aggfunc='count').reset_index()

df_splunk_subs_charge

,Reseller,SubscriberId,Charge,ARPU
0,DENT,1836473,5.620834,ARPU > 5 and <= 10 USD
1,DENT,1836474,0.001675,ARPU <= 1 USD
2,DENT,1836475,0.710836,ARPU <= 1 USD
3,DENT,1836477,0.124466,ARPU <= 1 USD
4,DENT,1836478,0.051165,ARPU <= 1 USD
...,...,...,...,...
9516,X2One,1976272,31.097127,ARPU > 10 USD
9517,X2One,2026541,31.032354,ARPU > 10 USD
9518,X2One,2026542,10.625663,ARPU > 10 USD
9519,X2One,2026544,0.040000,ARPU <= 1 USD


In [759]:
# Step 2. Report from OCS of Active subscribers

sql_srt = """
SELECT
r.RESELLER_NAME AS Reseller
,ss.STATUS AS Status
,COUNT(*) AS TotalSubs
FROM RESELLERS r
INNER JOIN ACCOUNTS ra
ON r.ACCOUNT_ID = ra.ACCOUNT_ID
INNER JOIN ACCOUNTS a
ON r.RESELLER_ID = a.RESELLER_ID
INNER JOIN SUBSCRIBERS sr
ON sr.ACCOUNT_ID = a.ACCOUNT_ID
INNER JOIN SUBSCRIBER_STATUS ss
ON ss.SUBSCRIBER_ID = sr.SUBSCRIBER_ID
WHERE
r.deleted is NULL
AND ss.END_DATE IS NULL
AND ss.status = 'Active'
GROUP BY
r.RESELLER_NAME
,ss.STATUS
ORDER BY r.RESELLER_NAME
"""
with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
    df_ocs_active_subs = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
df_ocs_active_subs.head()

,Reseller,Status,TotalSubs
0,1OT,Active,5
1,Axis Mobile Ventures LLC,Active,20
2,Cellact,Active,4993
3,CITICTEL RT,Active,2
4,Clay RT,Active,231


In [736]:
# Step 3. Merge 

pd.merge(df_ocs_active_subs, df_splunk_subs_charge, how='inner', on='Reseller')\
.to_csv(join(downloads, 'ResellerArpuJuly2020.csv'), index=False)

In [761]:
pd.merge(df_bss_subs_charge, df_ocs_active_subs, how='left', on='Reseller')\
.to_csv(join(downloads, 'ResellerArpuJuly2020.csv'), index=False)

In [764]:
0.24 * 7000

1680.0

In [ ]:
%reset -f